# Setup

In [ ]:
#!python3 -m pip install -U matplotlib
#!pip install -U scikit-learn
#!pip install ipywidgets

In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.seasonal import seasonal_decompose


from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.multioutput import RegressorChain
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
from ipywidgets import interactive,widgets, interact, HBox, Layout,VBox
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## Utils

In [ ]:
#!pip install statsmodels

In [2]:
""" Credit to Prof. Milad Toutounchian (Drexel University) for plotseasonal and seas_deco functions.
    Those functions were taken from DSCI 631 - Applied Machine Learning Timeseries notebooks
"""
def plotseasonal(res, axes, suptitle):
    res.observed.plot(ax=axes[0])
    axes[0].set_ylabel('Observed', fontsize=12)
    axes[0].set_title(suptitle, fontsize=12)
    res.trend.plot(ax=axes[1])
    axes[1].set_ylabel('Trend', fontsize=12)
    res.seasonal.plot(ax=axes[2])
    axes[2].set_ylabel('Seasonal', fontsize=12)
    res.resid.plot(ax=axes[3])
    axes[3].set_ylabel('Residual', fontsize=12)

def seas_deco(df):
    # Additive Decomposition
    result_add = seasonal_decompose(df['Count'], model='additive', extrapolate_trend='freq')
    
    fig, axes = plt.subplots(ncols=2, nrows=4, sharex=True, figsize=(18,5))

    plotseasonal(result_add, axes[:,0], 'Additive Decompose')
    
    
    try:
        # Multiplicative Decomposition 
        result_mul = seasonal_decompose(df['Count'], model='multiplicative', extrapolate_trend='freq')
        plotseasonal(result_mul, axes[:,1], 'Multiplicative Decompose')
    except ValueError:
        print("Multiplicative Decompistion Error: Not Appropriate for zero or negative values")
    
    plt.show()
        

In [26]:
def create_timeseries_data(county = ""):
    incidents_df = pd.read_csv('../data/Aggregated/incidents.csv')
    
    if county:
        incidents_df = incidents_df[incidents_df['Incident County Name'] == county]
    
    incidents_df['Month'] = incidents_df['Incident Date'].apply(lambda x: pd.Timestamp(datetime.strptime(x,"%m/%d/%Y")).month)
    
#     incidents_df["Fentanyl"] = incidents_df["All Drugs"].apply(lambda x: 1 if "FENTANYL" in x else 0)
    
#     incidents_df["Heroin"] = incidents_df["All Drugs"].apply(lambda x: 1 if "HEROIN" in x else 0)
    
    incidents_df["Year"] = incidents_df["Incident Date"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y").year)
    
    incidents_df['Count'] = 1

    return incidents_df


"""Credit to Marco Peixiero:
https://towardsdatascience.com/the-complete-guide-to-time-series-forecasting-using-sklearn-pandas-and-numpy-7694c90e45c1
"""
def window_input_output(input_length: int, output_length: int, data: pd.DataFrame) -> pd.DataFrame:
    
    df = data.copy()
    
    # Create x-features
    for i in range(1, input_length):
        df[f'x_{i}'] = df['Count'].shift(-i)
    
    # Create y-targets
    for i in range(output_length):
        df[f'y_{i}'] = df['Count'].shift(-output_length-i)
    
    # Drop any rows with NaN to prevent errors 
    df = df.dropna(axis=0)
    
    return df

def split_data(df, forecast = 5, test_size = 1, first_run = False):
    
    if first_run:
        # Create lags 
        final_df = window_input_output(forecast, forecast, df)
    else:
        final_df = df
        
    # Extract features and targets
    feature_cols = [col for col in final_df.columns if col.startswith('x') or col == "Count"]
    target_cols = [col for col in final_df.columns if col.startswith('y')]
    
    # Split data according to test_size
    x_train = final_df[feature_cols][:-test_size].values
    y_train = final_df[target_cols][:-test_size].values

    x_test = final_df[feature_cols][-test_size:].values
    y_test = final_df[target_cols][-test_size:].values
    
    return x_train, x_test, y_train, y_test, final_df

def fill_missing_months(df,
                        start_year = 2018,
                        end_year = 2022,
                        start_month = 1,
                        end_month = 12):
    
    # Ensure input has both month and year
    if "Year" not in df.columns or "Month" not in df.columns:
        raise ValueError("Missing one or more columns")
    
    # Copy dataframe
    temp_df = df.copy()
    
    # Find months that are missing in the set of dataframe
    years = [i for i in range(start_year, end_year + 1)]
    months = [i for i in range(start_month, end_month + 1)]
    
    # Complete Set of all combinations of years and months in the range required
    combination_ls = set([y * 100 + m for y in years for m in months])
    
    # Set of months and years presented in the dataframe
    a = set(temp_df[["Year", "Month"]].apply(lambda x: x["Year"] * 100 + x["Month"], axis = 1).tolist())
    
    # Missing pieces
    missing_months = combination_ls - a
    
    # Append missing months to dataframe
    for period in missing_months:
        month = period % 100
        year = period // 100
        county = temp_df["County"].unique()[0]
        temp_df = temp_df.append({'County': county,
                                  'Year': year,
                                  'Month': month,
                                  'Count' : 0}, ignore_index = True)
    
    
    return temp_df.sort_values(by = ["Year", "Month"])


def df_with_timestamp(df):
    
    # Copy dataframe
    temp_df = df.copy()
    
    # Convert year and month into datetime of the first day of the month
    temp_df["Date"] = temp_df.apply(lambda x: datetime(year = x["Year"], month = x["Month"], day = 1), axis = 1)
    
    # Return the new dataframe with date as index
    return temp_df.set_index("Date")


def plot_forecast(orig_df,
                  final_df,
                  x_train,
                  x_test,
                  y_train,
                  y_test,
                  fitted_estimators = [],
                  forecast = 5):
    
    ts_df = df_with_timestamp(orig_df[["Year", "Month", "Count"]])
    test_timestamp = [ts_df.index.tolist()[-forecast* 2 +1: -forecast], ts_df.index.tolist()[-forecast: ]]
    
    input_ts_ls = ts_df.iloc[:len(x_train) + 1,].index.tolist()
    input_ts_ls.extend(test_timestamp[0])

    input_count_ls = [i[0] for i in x_train]
    input_count_ls.extend(x_test[0])
    
    # Plot data 
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(input_ts_ls, input_count_ls, marker='P', color='blue', label='Input')
    ax.plot(test_timestamp[1], y_test[0], marker='P', color='red', label='Actual')
    
    # Plot predicted values for each estimator
    for est in fitted_estimators:
        name = str(est.estimators_[0].__class__).split(".")[-1][:-2]
        preds = est.predict(x_test)
        ax.plot(test_timestamp[1], preds[0], marker='P', label=f'{name}')
        print(f"RMSE {name}: {sqrt(mean_squared_error(y_test, preds)):.2f}")
    
    plt.legend()
    
    
def append_row(df):
    
    # Create a copy of df and last row in df
    temp_df = df.copy()
    temp_row = temp_df.iloc[-1:, :].copy()
    
    # Shift all values to the left
    for i in range(1,temp_row.shape[1]):
        temp_row.iloc[0,i - 1] = temp_row.iloc[0,i]
    
    # Set last column to NaN (to be predicted)
    temp_row.iloc[0, -1] = np.nan
    
    # Replace timestamp with the following month
    temp_row = temp_row.rename(index={temp_row.index[0]: temp_row.index[0] + relativedelta(months = 1)})
    
    # Append new row to dataframe
    temp_df = temp_df.append(temp_row)
    
    return temp_df

def estim(est, **kwargs):
    
    # Default estimator if none exist
    if not est:
        rfr_seq =  GradientBoostingRegressor(loss = "absolute_error",
                                            learning_rate = 0.1)
        return RegressorChain(rfr_seq)
    
    # Initialize estimator according to the input and arguments
    else:
        if "Random" in est:
            return RegressorChain(RandomForestRegressor(**kwargs))

        if "Decision" in est:
            kwargs_temp = {key:value for key,value in kwargs.items() if key != "n_estimators"}
            return RegressorChain(DecisionTreeRegressor(**kwargs_temp))

        if "Gradient" in est:
            return RegressorChain(GradientBoostingRegressor(**kwargs))
        else:
            raise ValueError("Estimator not supported")


def pred_to_year(est,
                 orig_df,
                 year = 2024,
                 month = 1,
                 forecast = 5,
                 test_size = 1,
                 **kwargs):
    
    # Create data with lags, and split into train and test
    (x_train,
     x_test,
     y_train,
     y_test,
     temp_df) = split_data(orig_df, forecast = forecast, test_size = test_size, first_run = True)
    
    # Convert indices to date timestamps
    temp_df = df_with_timestamp(temp_df)
    temp_df.drop(["County", "Year", "Month"], axis = 1, inplace = True)
    
    # Initialize helping variables
    index = 0
    last_train_index = len(temp_df) - 1 + forecast * 2
    threshold = datetime(year = year, month = month, day = 1)
    
    # Iterate till threshold date is reached
    while temp_df.iloc[-1:,:].index[0] < threshold:
        
        # Ensure not first run of the loop
        if index != 0:
            
            # Append a new row with NaN in the last y-column (to be forecasted)
            temp_df = append_row(temp_df)
            
            # Create data with lags, and split into train and test
            (x_train,
             x_test,
             y_train,
             y_test,
             _) = split_data(temp_df, forecast = forecast, test_size = 1)
            
        # Initialize estimator
        estimator = estim(est, **kwargs)
        
        # Fit & Predict
        estimator.fit(x_train, y_train)
        preds = estimator.predict(x_test)
        
        # Add the forecast to the last cell in dataframe
        temp_df.iloc[-1, -1] = preds[0][-1]
        
        # Increase loop index
        index += 1
    
    return temp_df, last_train_index

def forecast_county(df,
                    county = "Philadelphia",
                    est = "GradientBoostingRegressor",
                    forecast_size = 5,
                    **kwargs):
    
    forecast_df, last_train_index = pred_to_year(est, orig_df = df[:-forecast_size], **kwargs)
    
    fig, ax = plt.subplots(figsize=(10, 5))
    print(f"RMSE: {mean_squared_error(df.iloc[last_train_index:last_train_index + forecast_size]['Count'], forecast_df.iloc[last_train_index:last_train_index + forecast_size]['Count'])}" )
    plt.title(f"Forecast for {county} to {kwargs['year'] if 'year' in kwargs.keys() else '2024'} using {est}")    
    
    # Plot Input (Training set)
    ax.plot(forecast_df.iloc[:last_train_index].index, forecast_df.iloc[:last_train_index]["Count"], marker='P', color='blue', label = "Train")
    
    # Plot Test set
    ax.plot(forecast_df.iloc[last_train_index : last_train_index + 5].index, df.iloc[last_train_index : last_train_index + 5]["Count"], marker='P', color='red', label = "Test")
    
    # Plot forecast
    ax.plot(forecast_df.iloc[last_train_index -1:last_train_index + 1].index, forecast_df.iloc[last_train_index -1 :last_train_index + 1]["Count"], '--', color='orange')
    ax.plot(forecast_df.iloc[last_train_index:].index, forecast_df.iloc[last_train_index:]["Count"], marker='P', color='orange', label = "Forecast")
    
    # Show graph and legend
    ax.legend()
    plt.show()
    
    # Plot decomposition
    seas_deco(forecast_df.iloc[:last_train_index])

# Interactive Plot

In [27]:

print("Reading Dataset")
time_series_df = create_timeseries_data()
ls = ["Incident County Name",
  "Month",
  "Year",
  "Count"]

print("Preprocessing")
time_series_df = time_series_df[ls]
time_series_df = time_series_df.groupby(['Incident County Name',
                                         'Year',
                                         'Month']).sum().reset_index()
time_series_df.rename(columns={'Incident County Name': 'County'}, inplace=True)

Reading Dataset
Preprocessing


In [28]:
# Create list of counties
temp_df = pd.read_csv('../data/Aggregated/incidents.csv')
counties_ls = temp_df["Incident County Name"].unique().tolist()
del temp_df

# Create county dropdown
county_dropdown = widgets.Dropdown(
    options=counties_ls,
    value='Philadelphia',
    description='County:',
    disabled=False,
)

# Create regressor dropdown
regressor_dropdown = widgets.Dropdown(
    options=["Gradient Boosting", "Random Forest", "Decision Tree"],
    value='Gradient Boosting',
    description='Regressor:',
    disabled=False,
)

# Month and Year sliders
year_slider = widgets.IntSlider(
    value=2024,
    min=2023,
    max=2026,
    step=1,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

month_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=12,
    step=1,
    description='Month:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


# Hyper parameter tuning
criterion_dropdown = widgets.Dropdown(
    options=["friedman_mse", "squared_error"],
    value='friedman_mse',
    description='Criterion:',
    disabled=False,
)

max_features_dropdown = widgets.Dropdown(
    options=["auto", "log2", "sqrt"],
    value='auto',
    description='Max Features:',
    disabled=False,
)

n_estimators_slider = widgets.IntSlider(
    value=1,
    min=2,
    max=1000,
    step=1,
    description='N Estimators:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [29]:
def f(county,
      year,
      regressor,
      month,
      criterion,
      n_estimators,
      max_features
      ):
    temp_df = time_series_df[time_series_df["County"] == county]
    temp_df = fill_missing_months(temp_df)
    temp_df = temp_df[:-1] # Removing 2023
    if "Decision" in regressor:
        n_estimators_slider.disabled = True
    else:
        n_estimators_slider.disabled = False
    forecast_county(temp_df,
                    county = county,
                    est = regressor,
                    year = year,
                    month = month,
                    criterion = criterion,
                    n_estimators = n_estimators,
                    max_features = max_features,
                    random_state = 42
                   )

interactive_plot = interactive(f,
                               county = county_dropdown,
                               regressor = regressor_dropdown,
                               year = year_slider,
                               month = month_slider,
                               criterion = criterion_dropdown,
                               n_estimators = n_estimators_slider,
                               max_features = max_features_dropdown
                               )

controls = HBox(interactive_plot.children[:-1], layout = Layout(flex_flow='row wrap', width='85%'))
output = interactive_plot.children[-1]
display(VBox([controls, output]))

# Testing Functions

### Reading Dataset

In [ ]:
time_series_df = create_timeseries_data()

### Data preprocessing

In [ ]:
ls = ["Incident County Name",
      "Month",
      "Year",
      "Count"]

time_series_df = time_series_df[ls]
time_series_df = time_series_df.groupby(['Incident County Name',
                                         'Year',
                                         'Month']).sum().reset_index()
time_series_df.rename(columns={'Incident County Name': 'County'}, inplace=True)

time_series_df = fill_missing_months(time_series_df)

# Sanity Check
# test[(test['County'] == 'Philadelphia')]

### Time Series Forecasting

In [ ]:
forecast = 5
test_size = 1
(x_train,
 x_test,
 y_train,
 y_test,
 final_df) = split_data(time_series_df[:-1], forecast = forecast, test_size = test_size, first_run = True) # -1 is to remove Jan 2023

### Decision Tree Regressor

In [ ]:
dtr_seq = DecisionTreeRegressor(criterion = "absolute_error",
                                max_depth = 1,
                                random_state=42)
chained_dtr = RegressorChain(dtr_seq)
chained_dtr.fit(x_train, y_train)
dtr_seq_preds = chained_dtr.predict(x_test)

sqrt(mean_squared_error(y_test, dtr_seq_preds))

### Random Forest Regressor

In [ ]:
rfr_seq = RandomForestRegressor(criterion = "friedman_mse",
                                max_depth = 1,
                                random_state = 42)
chained_rfr = RegressorChain(rfr_seq)
chained_rfr.fit(x_train, y_train)
rfr_seq_preds = chained_rfr.predict(x_test)

sqrt(mean_squared_error(y_test, rfr_seq_preds))

In [ ]:

gbr_seq = GradientBoostingRegressor(loss = "absolute_error",
                                    learning_rate = 0.1,
                                    random_state=42)
chained_gbr = RegressorChain(gbr_seq)
chained_gbr.fit(x_train, y_train)
gbr_seq_preds = chained_gbr.predict(x_test)
sqrt(mean_squared_error(y_test, gbr_seq_preds))

### Plots

In [ ]:
plot_forecast(orig_df = time_series_df[:-1],
              final_df = final_df,
              x_train = x_train,
              x_test = x_test,
              y_train = y_train,
              y_test = y_test,
              fitted_estimators = [chained_rfr,
                                   chained_gbr,
                                   chained_dtr])

### Prediction to 2024

In [ ]:
rfr_seq =  GradientBoostingRegressor(loss = "absolute_error",
                                    learning_rate = 0.1)
chained_rfr = RegressorChain(rfr_seq)

_ = pred_to_year(LinearRegression(), time_series_df[:-1])
_

In [ ]:
_ = pred_to_year("Gradient", orig_df = time_series_df[:-1])
_

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Prediction to 2024 using Gradient Boosting Regression")
ax.plot(_.index, _["Count"], marker='P', color='red')

In [ ]:
_ = pred_to_year("Random Forest", criterion = "friedman_mse", n_estimators = 200, orig_df= time_series_df[:-1])
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Prediction to 2024 using Random Forest Regression")
ax.plot(_.index, _["Count"], marker='P', color='blue',)

In [ ]:
_ = pred_to_year("Decision Forest", criterion = "friedman_mse", max_features = "log2", orig_df= time_series_df[:-1])
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Prediction to 2024 using Decision Tree Regression")
ax.plot(_.index, _["Count"], marker='P', color='blue',)

In [ ]:
_ = pred_to_year("Decision Forest", criterion = "friedman_mse", max_features = "log2", orig_df= time_series_df[:-2])
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Prediction to 2024 using Decision Tree Regression")
ax.plot(_.index, _["Count"], marker='P', color='blue',)

In [ ]:
_ = pred_to_year("Gradient", orig_df = time_series_df[:-10])
fig, ax = plt.subplots(figsize=(10, 5))
plt.title("Prediction to 2024 using Decision Tree Regression")
ax.plot(_.index, _["Count"], marker='P', color='blue')

In [ ]:
forecast_county(county = "Allegheny")